## **Building an Agent with search tool**

In [102]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.agents import AgentType, initialize_agent, load_tools

import getpass
import os
from dotenv import load_dotenv

load_dotenv()

True

In [103]:
# Load the API key from the environment
def get_mistral_api_key():
    key = os.getenv("MISTRAL_API_KEY")
    if not key:
        key = getpass.getpass("Enter your Mistral API key: ")
        os.environ["MISTRAL_API_KEY"] = key
    return key

# Get the Mistral API key
mistral_api_key = get_mistral_api_key()

In [104]:
# Load the API key from the environment
def get_tavily_api_key():
    key = os.getenv("TAVILY_API_KEY")
    print(f"Key from environment: {'Found' if key else 'Not found'}")
    if not key:
        print("Please enter your Tavily API key when prompted:")
        key = getpass.getpass("Enter your TAVILY API key: ")
        os.environ["TAVILY_API_KEY"] = key
    return key

# Get the TAVILY API key
tavily_api_key = get_tavily_api_key()

Key from environment: Found


In [105]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Think step by step before responding."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [106]:
llm = ChatMistralAI(model="mistral-large-latest", api_key=mistral_api_key)
search = TavilySearchResults(api_key=tavily_api_key)
python = PythonREPLTool()

In [115]:
tools = [search, python]

In [116]:
message_history = ChatMessageHistory()

In [117]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [118]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [119]:
agent1 = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [112]:
agent1.invoke({"input": "hi"}, config={"configurable": {"session_id": "session1"}})



> Entering new AgentExecutor chain...
Hello! How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!

> Finished chain.


{'input': 'hi',
 'chat_history': [],
 'output': 'Hello! How can I assist you today? If you have any questions or need help with something specific, feel free to let me know!'}

In [120]:
response=agent1.invoke({"input": "When is the ICC Men's T20 2024 World Cup scheduled?"},config={"configurable": {"session_id": "session1"}})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': "ICC Men's T20 2024 World Cup scheduled?"}`


[{'url': 'https://www.olympics.com/en/news/t20-cricket-world-cup-2024-schedule-results-scores-standings-full-list', 'content': 'Nassau County International Cricket Stadium, New York - Sri Lanka 77 (19.1 overs), South Africa 80/4 (16.2 overs): South Africa won by 6 wickets Nassau County International Cricket Stadium, New York - Ireland 96 (16 overs), India 97/2 (12.2 overs): India won by 8 wickets Nassau County International Cricket Stadium, New York - India 119 (19 overs), Pakistan 113/7 (20 overs): India won by 6 runs Nassau County International Cricket Stadium, New York - South Africa 113/6 (20 overs), Bangladesh 109/7 (20 overs): South Africa won by 4 runs Nassau County International Cricket Stadium, New York - USA 110/8 (20 overs), India 111/3 (18.2 overs): India won by 7 wickets'}, {'url': 'https://www.cricbuzz.com/cricket-series/7476/icc-me

In [114]:
print(response['output'])

["The ICC Men's T20 World Cup 2024 is scheduled to take place in June 2024. Specific matches include:\n\n- 52nd Match, Super Eights, Group 1 in Kingstown on June 24, 2024\n- 50th Match, Super Eights, Group 2 in North Sound on June 23, 2024\n- 49th Match, Super Eights, Group 2 in Bridgetown on June 23, 2024\n- 48th Match, Super Eights, Group 1 in Kingstown on June 22, 2024\n- 47th Match, Super Eights, Group 1 in North Sound on June 22, 2024\n- 46th Match, Super Eights, Group 2 in Bridgetown on June 21, 2024 ", {'type': 'reference', 'reference_ids': [1]}, '.']


In [121]:
response = agent1.invoke({"input": "Today's date is May 1st, 2024. How many days befor the first match starts?"}, config={"configurable": {"session_id": "session1"}})






> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `{'query': 'from datetime import datetime\n\n# Define the dates\ncurrent_date = datetime(2024, 5, 1)\nevent_date = datetime(2024, 6, 13)\n\n# Calculate the difference\ndifference = event_date - current_date\n\n# Print the number of days\nprint(difference.days)'}`


43
There are 43 days until the first match starts on June 13, 2024.

> Finished chain.


In [122]:
print(response['output'])

There are 43 days until the first match starts on June 13, 2024.
